In [3]:
import numpy as np
from numba import njit, prange
from scipy.optimize import differential_evolution
from functions import is_target_blocked, v_of_fy

In [4]:
delta_t = 0.01  # time step(s)

M1_start_pos = np.array([20000, 0, 2000])
FY1_start_pos = np.array([17800, 0, 0])
FY1_height = np.array([0, 0, 1800])  # the height of FY1(m)

Vm = 300 * -M1_start_pos / np.linalg.norm(M1_start_pos)  # speed of missiles(m/s)

g = np.array([0, 0, -9.8])  # the gravity acceleration
V_cloud = np.array([0, 0, -3])  # the velocity of the cloud(m/s)


In [5]:
from scipy.optimize import differential_evolution
@njit(fastmath=True, parallel=True, cache=True, nogil=True)
def objective(x):
    theta, speed, t_11, t_12, t_21, t_22, t_31, t_32 = x
    V1 = v_of_fy(speed, theta)
    M1_end_pos = M1_start_pos + Vm * t_11
    T1 = np.sqrt(np.sum(M1_end_pos**2)) / 300    
    steps = int(T1 / delta_t)
    cloud_center_1 = FY1_start_pos + FY1_height + V1 * (t_11 + t_12) + 0.5 * g * t_12**2
    cloud_center_2 = FY1_start_pos + FY1_height + V1 * (t_11 + t_21 + t_22) + 0.5 * g * t_22**2
    cloud_center_3 = FY1_start_pos + FY1_height + V1 * (t_11 + t_21 + t_31 + t_32) + 0.5 * g * t_32**2
    counter = 0
    for i in prange(steps):
        t = i * delta_t
        M1_pos_now = M1_end_pos + Vm * t
        cloud_center_1_now = cloud_center_1 + V_cloud * (t - t_12)
        cloud_center_2_now = cloud_center_2 + V_cloud * (t - t_21 - t_22)
        cloud_center_3_now = cloud_center_3 + V_cloud * (t - t_21 - t_31 - t_32)
        if (is_target_blocked(M1_pos_now, cloud_center_1_now, t - t_12) + is_target_blocked(M1_pos_now, cloud_center_2_now, t - t_21 - t_22) + is_target_blocked(M1_pos_now, cloud_center_3_now, t - t_21 - t_31 - t_32)):
            counter += 1

    return -counter * delta_t

In [63]:
# pack the arguments into a single tuple for the objective function

bounds = [(0, 2*np.pi), (70, 140), (0, 5), (0, 5), (1, 10), (0, 10), (1, 10), (0, 10)]
result = differential_evolution(objective, bounds, popsize=32, polish=True, disp=True)
theta, speed, t_11, t_12, t_21, t_22, t_31, t_32 = result.x
max_time = -result.fun
print(f"最长遮蔽时间: {max_time:.2f} s")
print(f"最优参数: theta={theta:.3f}, speed={speed:.3f}, t_11={t_11:.3f}, t_12={t_12:.3f}, t_21={t_21:.3f}, t_22={t_22:.3f}, t_31={t_31:.3f}, t_32={t_32:.3f}")

differential_evolution step 1: f(x)= -1.8
differential_evolution step 2: f(x)= -1.8
differential_evolution step 3: f(x)= -4.43
differential_evolution step 4: f(x)= -4.43
differential_evolution step 5: f(x)= -4.43
differential_evolution step 6: f(x)= -4.43
differential_evolution step 7: f(x)= -4.43
differential_evolution step 8: f(x)= -4.43
differential_evolution step 9: f(x)= -4.43
differential_evolution step 10: f(x)= -4.5200000000000005
differential_evolution step 11: f(x)= -4.5200000000000005
differential_evolution step 12: f(x)= -4.5200000000000005
differential_evolution step 13: f(x)= -4.5200000000000005
differential_evolution step 14: f(x)= -4.5200000000000005
differential_evolution step 15: f(x)= -4.5200000000000005
differential_evolution step 16: f(x)= -4.5200000000000005
differential_evolution step 17: f(x)= -4.5200000000000005
differential_evolution step 18: f(x)= -4.5200000000000005
differential_evolution step 19: f(x)= -4.5200000000000005
differential_evolution step 20: f(x


最长遮蔽时间: 7.57 s
最优参数: theta=3.136, speed=139.715, t_11=0.039, t_12=3.653, t_21=3.573, t_22=5.343, t_31=1.943, t_32=6.022

最长遮蔽时间: 7.59 s
最优参数: theta=3.135, speed=139.873, t_11=0.004, t_12=3.644, t_21=3.511, t_22=5.339, t_31=1.957, t_32=6.037

In [64]:
# calculate the time of single cloud's effective blocking
def calc_single_cloud_time(t_1, t_2, delta_t):
    # t_1: time before thrown，t_2: time between thrown and bomb
    cloud_center = FY1_start_pos + FY1_height + V1 * (t_1 + t_2) + 0.5 * g * t_2**2
    M1_end_pos = M1_start_pos + Vm * (t_1 + t_2)
    T1 = np.linalg.norm(M1_end_pos) / 300
    tmax = max(20.0, T1)
    steps = int(tmax / delta_t)
    counter = 0
    for i in range(steps):
        t = i * delta_t
        M1_pos_now = M1_end_pos + Vm * t
        cloud_center_now = cloud_center + V_cloud * t 
        if is_target_blocked(M1_pos_now, cloud_center_now, t):
            counter += 1
    return counter * delta_t

# 使用最优参数
V1 = v_of_fy(speed, theta)
time1 = calc_single_cloud_time(t_11, t_12, delta_t)
time2 = calc_single_cloud_time(t_11 + t_21, t_22, delta_t)
time3 = calc_single_cloud_time(t_11 + t_21 + t_31, t_32, delta_t)

print(f"第一颗烟雾弹有效遮蔽时长: {time1:.2f} s")
print(f"第二颗烟雾弹有效遮蔽时长: {time2:.2f} s")
print(f"第三颗烟雾弹有效遮蔽时长: {time3:.2f} s")

第一颗烟雾弹有效遮蔽时长: 3.69 s
第二颗烟雾弹有效遮蔽时长: 2.69 s
第三颗烟雾弹有效遮蔽时长: 1.26 s


In [65]:
from functions import fy_throw_position
# 第一颗烟雾弹
throw1_pos = fy_throw_position(FY1_start_pos, FY1_height, theta, speed, t_11)
# 第二颗烟雾弹
throw2_pos = fy_throw_position(FY1_start_pos, FY1_height, theta, speed, t_11 + t_21)
# 第三颗烟雾弹
throw3_pos = fy_throw_position(FY1_start_pos, FY1_height, theta, speed, t_11 + t_21 + t_31)

print(f"第一颗烟雾弹起爆点: {throw1_pos}")
print(f"第二颗烟雾弹起爆点: {throw2_pos}")
print(f"第三颗烟雾弹起爆点: {throw3_pos}")

第一颗烟雾弹起爆点: [1.77994942e+04 3.11159859e-03 1.80000000e+03]
第二颗烟雾弹起爆点: [1.73084329e+04 3.02403802e+00 1.80000000e+03]
第三颗烟雾弹起爆点: [1.70346388e+04 4.70837313e+00 1.80000000e+03]


In [66]:
from functions import fy_bomb_position

# 第一颗烟雾弹
bomb1_pos = fy_bomb_position(FY1_start_pos, FY1_height, theta, speed, t_11, t_12)
# 第二颗烟雾弹
bomb2_pos = fy_bomb_position(FY1_start_pos, FY1_height, theta, speed, t_11 + t_21, t_22)
# 第三颗烟雾弹
bomb3_pos = fy_bomb_position(FY1_start_pos, FY1_height, theta, speed, t_11 + t_21 + t_31, t_32)

print(f"第一颗烟雾弹起爆点: {bomb1_pos}")
print(f"第二颗烟雾弹起爆点: {bomb2_pos}")
print(f"第三颗烟雾弹起爆点: {bomb3_pos}")

第一颗烟雾弹起爆点: [1.72898715e+04 3.13822457e+00 1.73495086e+03]
第二颗烟雾弹起爆点: [1.65616481e+04 7.61813244e+00 1.66031974e+03]
第三颗烟雾弹起爆点: [1.61901878e+04 9.90329375e+00 1.62139524e+03]
